**22.5-1<br>How can the number of strongly connected components of a graph change if a new edge is added?**

The number is either unchanged or reduced by $1$. Suppose you add a new edge $(u,v)$: if the  $u$ was reachable from $v$ and $u$, $v$ belong to distinct SCC, they will be fused into one; if not the number of SCC will not changed.

**22.5-2<br>Show how the procedure STRONG-CONNECTED-COMPONENTS works on the graph of Figure 22.6. Specifically, show the finishing times computed in line 1 and the forest produced in line 3. Assume the loop of lines 5-7 of DFS consider vertices in alphabetical order and that the adjacency lists are in alphabetical order.**

Running `scc()` yields 5 SCC of Figure 22.6.

In [1]:
def transpose_list(G):
    # Gt is an empty list
    Gt={i:[] for i in G.keys()}
    # scan through G
    for u in G:
        for v in G[u]:
            Gt[v].append(u)
    return Gt
def dfs_ntree(G):
    global time # time is a global varaible, it is accumulative
    time=0
    colour={i:'white' for i in G} # initial colour of all vertices are white
    t_discover={}
    t_finish={}
    predecessor={}
    global ntree #number of connected component
    ntree=0
    cc={} #attribute u.cc
    for u in G:
        if colour[u]=='white':
            ntree+=1 # increment ntree for a start of a new dfs tree
            #cc[u]=ntree
            dfs_visit_ntree(G, u, colour, t_discover,t_finish,predecessor,cc) #recursion
    return cc#, t_finish#, colour, t_discover,predecessor 

def dfs_visit_ntree(G, u, colour, t_discover,t_finish,predecessor,cc):
    global time
    time+=1
    t_discover[u]=time # a white vertex is discovered
    colour[u]='grey' # and coloured to grey
    cc[u]=ntree # for every other tree inside the dfs, they have the same ntree
    for v in G[u]:
        if colour[v]=='white':
            predecessor[v]=u
            dfs_visit_ntree(G, v, colour, t_discover,t_finish,predecessor,cc)
    colour[u]='black'
    time+=1
    t_finish[u]=time  
    
from collections import deque
def topological_sort(G):
    global time # time is a global varaible, it is accumulative
    time=0
    colour={i:'white' for i in G} # initial colour of all vertices are white
    t_discover={}
    t_finish={}
    predecessor={}
    sorted_list=deque() # to store the sorted list
    for u in G:
        if colour[u]=='white':
            dfs_visit_topo(G, u, colour, t_discover,t_finish,predecessor,sorted_list) #recursion
    return sorted_list #,t_discover, t_finish

def dfs_visit_topo(G, u, colour, t_discover,t_finish,predecessor,sorted_list):
    global time
    time+=1
    t_discover[u]=time # a white vertex is discovered
    colour[u]='grey' # and coloured to grey
    for v in G[u]:
        if colour[v]=='white':
            predecessor[v]=u
            dfs_visit_topo(G, v, colour, t_discover,t_finish,predecessor,sorted_list)
    sorted_list.appendleft(u) # a vertex fully explored is appended on the left of the list
    colour[u]='black'
    time+=1
    t_finish[u]=time 
def scc(G):
    # compute tranpose of G
    Gt=transpose_list(G)
    
    # topological sorted vertices of G
    sorted_vertices=topological_sort(G)
    #print (sorted_vertices)
    
    # rearrange Gt according to topologically sorted vertices in G
    # this is done by merging GG, an empty dict "sorted" according to sorted_vertices
    # and Gt
    GG={i:[] for i in sorted_vertices}
    Gt={**GG, **Gt}
   
    # display dfs trees of Gt in order of sorted_vertices
    return dfs_ntree(Gt)

In [2]:
G={'q':['s','t','w'],
    'r':['u'],
    's':['v'],
    't':['x','y'],
    'w':['s'],
    'u':['y'],
    'v':['w'],
    'x':['z'],
    'y':['q'],
    'z':['x']}
scc(G)

{'r': 1,
 'u': 2,
 'q': 3,
 'y': 3,
 't': 3,
 'x': 4,
 'z': 4,
 's': 5,
 'w': 5,
 'v': 5}

**22.5-3<br>Professor Bacon claims that the algorithm for strongly connected components would be simpler if it used the original (instead of the tranpose) graph in the second DFS and scanned the vertices in order of *increasing* finishing times. Does the simple algorithm always produce correct results?**

It is not always correct. This is because the finishing times obtained from the first DFS on $G$ also depends on the order of vertices in $adj[u]$. Consider a counter example of a directed graph $G1$ with edges $(b,c)$, $(c,a)$ and $(c,b)$ which has two SCC: {a} and {c,b}.

Running `scc_bacon` on `G_order_correct` and `G_order_incorrect` yields different results, although they both represent $G1$.

The problem is that the second DFS starts from vertices with earliest finish time. However, Lemma 22.14 is true for latest finishing time, but not true for **earliest finishing time**. It is easy to get confused!

In [3]:
def scc_bacon(G):
    
    # topological sorted vertices of G, then reverse
    sorted_vertices=topological_sort(G)
    sorted_vertices.reverse()
    
    # rearrange G according to topologically reverse-sorted vertices in G
    # this is done by merging GG, an empty dict "sorted" according to sorted_vertices wigh G
    GG={i:[] for i in sorted_vertices}
    G={**GG, **G}
   
    # display dfs trees of G in order of sorted_vertices
    return dfs_ntree(G)

In [4]:
G_order_correct={'c':['a','b'],'b':['c'],'a':[],}
G_order_incorrect={'c':['b','a'],'b':['c'],'a':[],}
print(scc_bacon(G_order_correct))
print (scc_bacon(G_order_incorrect)) #gives only one SCC
print(scc(G_order_correct))
print (scc(G_order_incorrect))

{'a': 1, 'b': 2, 'c': 2}
{'b': 1, 'c': 1, 'a': 1}
{'c': 1, 'b': 1, 'a': 2}
{'c': 1, 'b': 1, 'a': 2}


**22.5-5<br>Give an $O(V+E)$-time algorithm to compute the component graph of a directed graph $G=(V,E)$. Make sure that there is at amost one edge between two vertices in the component graph your algorithm produces.**

Given that $((G^T)^{SCC})^T=G^{SCC}$, the `scc_graph` first computes $(G^T)^{SCC}$ then tranponses it to obtain $G^{SCC}$. To compute $(G^T)^{SCC}$, `dfs_ntree_graph` is slightly modified from `dfs_ntree` we defined in 22.5-2. The major change is the addition of line 33-35 in the recursive `dfs_visit_ntree_graph`:
* It scans the black vertices in a "descendant" SCC and add it as edge in SCC graph
* It does not scan the black vertices lying in the same SCC, to avoid having multiple edges between two vertices in the SCC graph

Running `scc_graph(G)` returns:
1. an adjacency list of SCC of $G$ (in numerical naming 1, 2, ...)
2. a reference dictionary to see to which SCC the vertices $V$ in $G$ belong.

In [5]:
def dfs_ntree_graph(G):
    global time # time is a global varaible, it is accumulative
    time=0
    colour={i:'white' for i in G} # initial colour of all vertices are white
    t_discover={}
    t_finish={}
    predecessor={}
    global ntree #number of connected component
    ntree=0
    cc={} #attribute u.cc
    graph={} # SCC graph, as adjacency list
    for u in G:
        if colour[u]=='white':
            ntree+=1 # increment ntree for a start of a new dfs tree
            graph[ntree]=[] # no. vertices in SCC graph equals to no. DFS trees in second DFS
            dfs_visit_ntree_graph(G, u, colour, t_discover,t_finish,predecessor,cc,graph) #recursion
    return graph,cc#, t_finish#, colour, t_discover,predecessor 

def dfs_visit_ntree_graph(G, u, colour, t_discover,t_finish,predecessor,cc,graph):
    global time
    time+=1
    t_discover[u]=time # a white vertex is discovered
    colour[u]='grey' # and coloured to grey
    cc[u]=ntree # for every other tree inside the dfs, they have the same ntree
    flagnext=10000 # set a very large number so that cc[v]<flagnext is true at start point
    for v in G[u]:
        if colour[v]=='white':
            predecessor[v]=u
            dfs_visit_ntree_graph(G, v, colour, t_discover,t_finish,predecessor,cc,graph)
        # 'black' means vertices in another SCC
        # cc[v]<flagnext so that the black vertices are not in the same SCC
        # ->at most one edge connecting two SCC
        elif colour[v]=='black' and cc[v]<flagnext: 
            graph[ntree].append(cc[v]) 
        flagnext=cc[v]
    colour[u]='black'
    time+=1
    t_finish[u]=time 
def scc_graph(G):
    # compute tranpose of G
    Gt=transpose_list(G)
    
    # topological sorted vertices of G
    sorted_vertices=topological_sort(G)
    #print (sorted_vertices)
    
    # rearrange Gt according to topologically sorted vertices in G
    # this is done by merging GG, an empty dict "sorted" according to sorted_vertices
    # and Gt
    GG={i:[] for i in sorted_vertices}
    Gt={**GG, **Gt}
   
    # dfs_ntree(Gt)[0] is the SCC graph of Gt, it is an adjacency list
    # we need to traponse it once of obtain SCC of G, because of 22.5-3
    # dfs_ntree(Gt)[1] shows which vertice belong to which SCC
    return transpose_list(dfs_ntree_graph(Gt)[0]), dfs_ntree_graph(Gt)[1]

In [6]:
G1={'a':['b'],
    'b':['c','e','f'],
    'c':['d','g'],
    'd':['c','h'],
    'e':['a','f'],
    'f':['g'],
    'g':['f','h'],
    'h':['h']}  
scc_graph(G1)

({1: [2, 3], 2: [3, 4], 3: [], 4: []},
 {'a': 1, 'e': 1, 'b': 1, 'c': 2, 'd': 2, 'g': 3, 'f': 3, 'h': 4})